# CAFQA

In [1]:
import sys
sys.path.append("../")
from clapton.clapton import claptonize
from clapton.ansatzes import circular_ansatz
import numpy as np

In [2]:
# define Hamiltonian, e.g. 3q Heisenberg model with random coefficients
paulis = ["XXI", "IXX", "YYI", "IYY", "ZZI", "IZZ"]
coeffs = np.random.random(len(paulis))

In [3]:
# define parametrized Clifford circuit that is being optimized over
# here we use the circular_ansatz template
# we fix 2q gates as they will not be optimized over
vqe_pcirc = circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True)

In [4]:
# the circuit consists of parametrized gates
for gate in vqe_pcirc.gates:
    print(gate.label, gate.is_fixed())

RY False
RY False
RY False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
RY False
RY False
RY False
RZ False
RZ False
RZ False


In [5]:
# non-fixed gates will be optimized over
# RY and RZ gates can assume 4 values k = 0,1,2,3 which descripe multiples of pi/2

In [6]:
# the initial parameters are all 0
vqe_pcirc.read()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [7]:
# we can look at the corresponding stim circuit
vqe_pcirc.stim_circuit().diagram()

q0: -I-I-X-@-I-I---
         | |
q1: -I-I-|-X-@-I-I-
         |   |
q2: -I-I-@---X-I-I-

In [8]:
# we can assign a different set of parameters
vqe_pcirc.assign([0,1,2,3,0,1,2,3,0,1,2,3])

In [9]:
vqe_pcirc.stim_circuit().diagram()

q0: -I------S_DAG-X-@-Y-S----------------
                  | |
q1: -SQRT_Y-I-----|-X-@-SQRT_Y_DAG-Z-----
                  |   |
q2: -Y------S-----@---X-I----------S_DAG-

In [10]:
# we can perform CAFQA by using the main optimization function "claptonize"
ks_best, _, energy_best = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

STARTING ROUND 0


started GA at id 1 with 1 procs

started GA at id 2 with 1 procs

started GA at id None with 1 procs



/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")
/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evo

started GA at id 3 with 1 procs



/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


[0, array([-3.4276615 , -1.71383075, -1.71383075,  0.        ]), array([2, 1, 2, 2, 2, 0, 0, 2, 2, 3, 0, 0], dtype=object)]
[0, array([-3.4276615 , -1.71383075, -1.71383075,  0.        ]), array([2, 1, 0, 2, 2, 0, 0, 1, 1, 0, 2, 0], dtype=object)]
[0, array([-3.4276615 , -1.71383075, -1.71383075,  0.        ]), array([1, 3, 3, 0, 2, 3, 3, 1, 3, 0, 0, 1], dtype=object)]
[0, array([-3.4276615 , -1.71383075, -1.71383075,  0.        ]), array([0, 3, 2, 1, 2, 2, 1, 3, 1, 0, 0, 2], dtype=object)]
[1, array([-3.4276615 , -1.71383075, -1.71383075,  0.        ]), array([2, 1, 2, 2, 2, 0, 0, 2, 2, 3, 0, 0], dtype=object)]
[1, array([-3.4276615 , -1.71383075, -1.71383075,  0.        ]), array([3, 3, 0, 2, 0, 1, 0, 3, 3, 2, 0, 2], dtype=object)]
[1, array([-3.4276615 , -1.71383075, -1.71383075,  0.        ]), array([1, 3, 3, 0, 2, 3, 3, 1, 3, 0, 0, 1], dtype=object)]
[1, array([-3.10379042, -1.55189521, -1.55189521,  0.        ]), array([0, 1, 1, 2, 2, 3, 0, 2, 0, 1, 0, 2], dtype=object)]
[2, arra

In [11]:
# the best parameters are
ks_best

[2, 1, 0, 3, 2, 0, 3, 1, 1, 0, 3, 1]

In [12]:
# with energy
energy_best

-1.7138307493071705

In [13]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.stim_circuit().diagram()

q0: -Y------S_DAG-X-@-SQRT_Y_DAG-I------------
                  | |
q1: -SQRT_Y-Z-----|-X-@----------SQRT_Y-S_DAG-
                  |   |
q2: -I------I-----@---X----------SQRT_Y-S-----

# noisy CAFQA (nCAFQA)

In [14]:
from clapton.depolarization import GateGeneralDepolarizationModel #TODO: check this out 

In [15]:
# let's add a noise model where we specify global 1q and 2q gate errors
nm = GateGeneralDepolarizationModel(p1=0.005, p2=0.02)

In [16]:
vqe_pcirc = circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True)
vqe_pcirc.add_depolarization_model(nm)

In [17]:
# after every gate a depol channel is added
vqe_pcirc.stim_circuit().diagram()

q0: -I-DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.02)-@-DEPOLARIZE2(0.02)---I-----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------
                                                                                                                         | |                 | |
q1: ---I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------|-|-----------------X-DEPOLARIZE2(0.02)-@-DEPOLARIZE2(0.02)-I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------
                                                                                                                         | |                                     | |
q2: ----------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)-@-DEPOLARIZE2(0.02)---------------------X-DEPOLARIZE2(0.02)--------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)-

In [18]:
# we can perform nCAFQA by using the main optimization function "claptonize"
# now with the noisy circuit
ks_best, energy_noisy, energy_noiseless = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

STARTING ROUND 0


started GA at id 1 with 1 procs

started GA at id 2 with 1 procs


started GA at id None with 1 procs

started GA at id 3 with 1 procs
[0, array([-1.87495981, -0.89632392, -0.9786359 ,  0.        ]), array([3, 0, 2, 2, 3, 1, 0, 3, 1, 2, 0, 2], dtype=object)]
[0, array([-2.91581936, -1.36392415, -1.55189521,  0.        ]), array([1, 3, 1, 2, 1, 1, 1, 1, 0, 3, 2, 1], dtype=object)]
[0, array([-3.23175191, -1.51792116, -1.71383075,  0.        ]), array([2, 1, 0, 3, 0, 3, 0, 3, 1, 1, 3, 3], dtype=object)]
[0, array([-3.24031748, -1.52648673, -1.71383075,  0.        ]), array([2, 1, 2, 2, 0, 0, 1, 2, 2, 3, 3, 1], dtype=object)]
[1, array([-3.21675391, -1.50292316, -1.71383075,  0.        ]), array([2, 1, 0, 1, 2, 1, 2, 1, 1, 3, 1, 3], dtype=object)]
[1, array([-1.86804781, -0.88941192, -0.9786359 ,  0.        ]), array([0, 0, 0, 3, 1, 1, 1, 1, 1, 3, 0, 2], dtype=object)]
[1, array([-3.2237751 , -1.50994435, -1.71383075,  0.        ]), array([2, 1, 2, 1, 2, 3, 3, 3, 3, 0, 

In [19]:
# the best parameters are
ks_best

[1, 3, 0, 0, 0, 2, 1, 3, 3, 0, 0, 2]

In [20]:
# with noisy/noiseless energy
energy_noisy, energy_noiseless

(-1.5212799891157378, -1.7138307493071705)

In [21]:
# differrence
np.abs(energy_noisy-energy_noiseless)

0.1925507601914327

In [22]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.snapshot_noiseless().circ_snapshot_noiseless.diagram()

q0: -SQRT_Y-----I-X-@-SQRT_Y-I------------
                  | |
q1: -SQRT_Y_DAG-I-|-X-@------SQRT_Y_DAG-I-
                  |   |
q2: -I----------Z-@---X------SQRT_Y_DAG-Z-